# CEO-REQUEST CHALLENGE

***CEO Question***

How do we increase customer satisfaction (so as to increase profit margins) while maintaining a healthy order volume?

**Findings from our Preliminary Analysis of the Orders and Sellers Datasets**


- The more sellers (`number_of_sellers`) and products (`number_of_products`) that are combined into a single order, the lower the order `review_score`.  
- The `wait_time` for the order, which is made up of two components (seller's `delay_to_carrier` + `carrier_delivery_time`) also has a significant association with low review scores.  
- When these features are examined on a per seller basis, we're can see that they still have an impact review score, and further we're able to identify which sellers are linked with lower review scores

While orders that contain more than a single seller or product do have a significant association with low review scores, these types of orders do not make up a large part of the orders on the platform, so any policy changes (e.g. limit orders to a single seller), would not have a large impact on the bottom line. 

Instead, we'll focus on identifying sellers who receive poor reviews and quantifying their impact on Olist's bottom line. By understanding the relationship, we'll be able to make a specific recommendation to the CEO on what steps to take to increase profit margins in the short term.  

**Next Steps**

In this Notebook, we'll investigate how much impact underperforming sellers have on the Olist's bottom line.

**Steps to our Analysis**
1. Identify how much revenue each seller brings in
2. Calculate the cost to Olist of bad reviews for each seller
3. Calculate impact on Olist bottom line if poor performers are removed from platform
4. Find out if reduced number of sellers will impact savings generated from operating at scale


**Problem statement**

> Should Olist remove underperforming sellers from its marketplace?

To analyse the impact of removing the worst sellers from Olist's marketplace, we can start with a what-if analysis: What would have happened if Olist had never accepted these sellers in the first place? For that:

Step ① Compute, for each `seller_id`, and cumulated since the beginning:
- The `revenues` it brings
- The `costs` associated with all its bad reviews
- The resulting `profits` (revenues - costs)
- The number of `orders` (it will impact overall IT costs)


Step ② We can then sort sellers by increasing profits for Olist, and for each number of sellers to remove, compute the financial impact it would have made had they never been accepted on the platform. We may find an optimal number of sellers to remove that maximizes Olist's profit margin

In [1]:
# Write down a detailed strategy (step by step) to create the DataFrame you need for step 1
# Think about how to re-use or update the logic you have already coded in your `olist` package

<details>
    <summary>Hints</summary>


Starting from your current `seller().get_training_data()` DataFrame:
- Can you easily transform it to compute Olist's positive `revenue_per_seller`? 
- Can you easily transform it to compute Olist's `cost_of_bad_reviews`?

❓Instead of starting again from scratch, investigate your source code in `seller.py` - how was the mean `review_score` per seller computed? Can you imagine a way to amend your code  to compute `cost_of_reviews` in the same process? 
</details>


Your turn!

# START

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from olist.seller import Seller
from olist.data import Olist 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
sellers = Seller().get_training_data()
sellers.head()

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,share_of_one_stars,share_of_five_stars,review_score,cost_of_reviews,n_orders,quantity,quantity_per_order,sales,revenues,profits,revenue,profits_before_it_costs
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,0.333333,0.333333,3.00,140,3,3,1.000,218.70,341.870,201.870,341.870,201.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,15.0,0.050000,0.725000,4.55,240,40,41,1.025,11703.07,2290.307,2050.307,2370.307,2130.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,1.0,0.000000,1.000000,5.00,0,1,1,1.000,158.00,15.800,15.800,95.800,95.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,1.0,0.000000,1.000000,5.00,0,1,1,1.000,79.99,7.999,7.999,87.999,87.999
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,1.0,1.000000,0.000000,1.00,100,1,1,1.000,167.99,16.799,-83.201,96.799,-3.201


In [4]:
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'share_of_one_stars', 'share_of_five_stars', 'review_score',
       'cost_of_reviews', 'n_orders', 'quantity', 'quantity_per_order',
       'sales', 'revenues', 'profits', 'revenue', 'profits_before_it_costs'],
      dtype='object')

In [5]:
sellers.rename(columns={'wait_time': 'seller_wait_time', 'review_score':'seller_review_score'}, inplace=True)

In [6]:
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'seller_wait_time', 'date_first_sale', 'date_last_sale',
       'months_on_olist', 'share_of_one_stars', 'share_of_five_stars',
       'seller_review_score', 'cost_of_reviews', 'n_orders', 'quantity',
       'quantity_per_order', 'sales', 'revenues', 'profits', 'revenue',
       'profits_before_it_costs'],
      dtype='object')

In [7]:
#export dataset for Tableau
path = '../data/csv/seller_dataset_modified.csv'
sellers.to_csv(path_or_buf=path, index=False)

## Revenue per selller

Olist generates revenue from its sellers from two components:
1. **Monthly Membership Fee** - sellers pay Olist an **80 BRL** (~$15 USD) monthly fee to use the platform
2. **Revenue Share** - for every transaction on the platform, Olist takes a **10% cut** on the product price (excl. shipping) for each item in an order


In [8]:
# Revenue Calculation: Monthly Membership Fee Calculation + Revenue Share

# Calculate total time on platform (in months)
number_of_months_on_olist = (sellers.date_last_sale - sellers.date_first_sale) / np.timedelta64(1, 'M')
sellers['months_on_olist'] = number_of_months_on_olist.map(lambda x: 1 if x < 1 else np.ceil(x))

sellers['revenue_per_seller'] = sellers.months_on_olist * 80 + sellers.sales * 0.1
sellers.head()

,seller_id,seller_city,seller_state,delay_to_carrier,seller_wait_time,date_first_sale,date_last_sale,months_on_olist,share_of_one_stars,share_of_five_stars,...,cost_of_reviews,n_orders,quantity,quantity_per_order,sales,revenues,profits,revenue,profits_before_it_costs,revenue_per_seller
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,0.333333,0.333333,...,140,3,3,1.000,218.70,341.870,201.870,341.870,201.870,341.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,15.0,0.050000,0.725000,...,240,40,41,1.025,11703.07,2290.307,2050.307,2370.307,2130.307,2370.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,1.0,0.000000,1.000000,...,0,1,1,1.000,158.00,15.800,15.800,95.800,95.800,95.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,1.0,0.000000,1.000000,...,0,1,1,1.000,79.99,7.999,7.999,87.999,87.999,87.999
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,1.0,1.000000,0.000000,...,100,1,1,1.000,167.99,16.799,-83.201,96.799,-3.201,96.799


In [9]:
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'seller_wait_time', 'date_first_sale', 'date_last_sale',
       'months_on_olist', 'share_of_one_stars', 'share_of_five_stars',
       'seller_review_score', 'cost_of_reviews', 'n_orders', 'quantity',
       'quantity_per_order', 'sales', 'revenues', 'profits', 'revenue',
       'profits_before_it_costs', 'revenue_per_seller'],
      dtype='object')

## Costs per seller

Because poor reviews by customers can have numerous costs to a business (e.g. customer support, bad word-of-mouth, low repeat rate, etc.), we use the following table as which estimates monetary costs associated with bad reviews.

Cost to Olist in Brazilian Reals (BRL) per bad review of seller:
- **1 star:** 100
- **2 stars:** 50
- **3 stars:** 40
- **4 stars:** 0
- **5 stars:** 0

In [10]:
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'seller_wait_time', 'date_first_sale', 'date_last_sale',
       'months_on_olist', 'share_of_one_stars', 'share_of_five_stars',
       'seller_review_score', 'cost_of_reviews', 'n_orders', 'quantity',
       'quantity_per_order', 'sales', 'revenues', 'profits', 'revenue',
       'profits_before_it_costs', 'revenue_per_seller'],
      dtype='object')

In [11]:
sellers[['seller_id','seller_review_score','cost_of_reviews']].head(5)

,seller_id,seller_review_score,cost_of_reviews
0,3442f8959a84dea7ee197c632cb2df15,3.00,140
1,d1b65fc7debc3361ea86b5f14c68d2e2,4.55,240
2,ce3ad9de960102d0677a81f5d0bb7b2d,5.00,0
3,c0f3eea2e14555b6faeea3dd58c1b1c3,5.00,0
4,51a04a8a6bdcb23deccc82b0b80742cf,1.00,100


In [12]:
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'seller_wait_time', 'date_first_sale', 'date_last_sale',
       'months_on_olist', 'share_of_one_stars', 'share_of_five_stars',
       'seller_review_score', 'cost_of_reviews', 'n_orders', 'quantity',
       'quantity_per_order', 'sales', 'revenues', 'profits', 'revenue',
       'profits_before_it_costs', 'revenue_per_seller'],
      dtype='object')

In [13]:
new_date = Seller().get_training_data()
new_date.head(20)
new_date.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'share_of_one_stars', 'share_of_five_stars', 'review_score',
       'cost_of_reviews', 'n_orders', 'quantity', 'quantity_per_order',
       'sales', 'revenues', 'profits', 'revenue', 'profits_before_it_costs'],
      dtype='object')

In [14]:





# Calculating per seller cost to Olist due to bad reviews
sellers['cost_of_review'] = sellers.review_score.map({ 1: 100, 2: 50, 3: 40, 4: 0, 5: 0})
sellers['review_cost_per_seller'] = sellers.groupby(by=['seller_id'], as_index=False).sum().cost_of_review




AttributeError: 'DataFrame' object has no attribute 'review_score'

In [ ]:
plt.figure(figsize=(12,8))
sns.set_style('darkgrid')
ax = sns.histplot(sellers.profits_before_it_costs)
ax.set_xlim(xmin=-5000, xmax=5000)
ax.set_xlabel('Profits before IT costs (BRL)')
ax.set_ylabel('Number of Sellers')
ax.set_title('Distribution of Seller Contribution to Olist Revenues')

In [ ]:
# Barplot to show number of sellers dragging on Olist that can be removed
def olist_impact(x):
    if x < 0:
        return 'negative'
    else:
        return 'positive'

sellers['impact'] = sellers['profits_before_it_costs'].apply(olist_impact)
sellers['impact'].value_counts()

In [ ]:
ax1 = sellers['impact'].value_counts().plot.bar()
ax1.set_ylabel('Count')
ax1.set_xlabel('Impact on Olist')

In [ ]:
sellers['profits_before_it_costs'].describe()

In [ ]:
# 75% of sellers on Olist platform contribute less than ~720 BRL (~USD=138) to Olist revenues

In [ ]:
sellers.sort_values('profits_before_it_costs', ignore_index=True).profits_before_it_costs.plot(figsize=(10,6));



# STEP 2: SORT SELLERS
Step ② We can then sort sellers by increasing profits for Olist, and for each number of sellers to remove, 
compute the financial impact it would have made had they never been accepted on the platform. 
We may find an optimal number of sellers to remove that maximizes Olist's profit margin


In [ ]:
# Sort sellers by how much profit each contributes to Olist
sellers.sort_values('profits_before_it_costs', ignore_index=True, inplace=True, ascending=False)

# number of sellers that are a cost (negative profits) on Olist
sellers_neg_profits = sellers.loc[sellers['olist_profits']<0].shape[0]
sellers_neg_profits

In [ ]:
#CALC Cumulative sum column of profits to Olist from each seller
sellers['accumulated_profits'] = sellers['profits_before_it_costs'].cumsum()
sellers.tail(10)

In [ ]:
#Total profits
profits = sellers['accumulated_profits']
profits.iloc[-1]

In [ ]:
#Profits per number of sellers removed [DO AFTER FACTOR IN IT COSTS]


In [ ]:
sellers.head(5)

In [ ]:
#IT Costs 
#Proportionality coefficient(B) of 0.028 based on integral of B*(n_orders)**1/2 = 500000
sellers['accumulated_orders'] = sellers['n_orders'].cumsum()
sellers['it_cost'] = (sellers['accumulated_orders']**0.5)*0.028
sellers.tail(5)

In [ ]:
#IT Unit Cost

sellers['it_unit_cost'] = sellers['it_cost'].diff()




In [ ]:
#Profits per number of sellers removed [DO AFTER FACTOR IN IT COSTS]
sellers['olist_profits'] = sellers['profits_before_it_costs'] - sellers['it_cost']
sellers.loc[2680:2690]

In [ ]:
sellers['olist_profits'][10:700].plot(figsize=(15,8));

In [ ]:
sellers['olist_profits'].plot(figsize=(15,8));

In [ ]:
sellers.loc[50:100, :]

In [ ]:
sellers['n_orders'].sort_values()

# Recommendation: 

## Remove poorest performing sellers on Olist platform for immediate profit increase

## Profit Analysis

- 280 poorest-performing sellers have negative impact on profits
- Top 100 highest performing sellers comprise bulk of profits

In [ ]:
# If sellers ordered by level of contribution to Olist profits

sellers['olist_profits'].plot(figsize=(15,7))
plt.xlabel('Number of Sellers')
plt.ylabel('Profit (BRL)')
plt.show()


## Profit Analysis

- Immediate profit increase of 320K BRL

In [ ]:
x_values = ['Before', 'After']
y_values = [1.25, 1.57]
plt.bar(x_values, y_values)
plt.ylabel('Profit (Millions of BRL)')
plt.ylim(0, 2)
plt.show()


## IT Cost Concerns 

- Increase in marginal costs due to drop in sellers negligible
- Some poorest-performing sellers contribute to larger share of IT cost burden


In [ ]:
sellers['it_cost'].plot(figsize=(15,7))
plt.xlabel('Number of Sellers')
plt.ylabel('IT Cost per Seller (BRL)')
plt.show()


In [ ]:
sellers['it_cost'][10:700].plot()

In [ ]:
ax1 = sellers['it_cost'].plot()
ax1.set(xlim=[0,3000], ylim=[-1000,2000])

ax2 = sellers['olist_profits'].plot()
ax2.set(xlim=[0,3000], ylim=[-1000,2000])
plt.xlabel['']




## Areas to Explore Further

- Evaluate whether can improve average seller performance
- Explore reviews for cause of dissatisfaction